In [1]:
import csv
import subprocess
import math
import json
import os
import shlex
import cv2 
import shutil
from tqdm import tqdm
import numpy as np

In [24]:
def cleardir(dirpath):
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)
    try:
        os.mkdir(dirpath)
    except OSError:
        print ("Creation of the directory %s failed" % dirpath)
        print(OSError)
    else:
        print ("Successfully created the directory %s " % dirpath)

In [25]:
# https://github.com/c0decracker/video-splitter/blob/master/ffmpeg-split.py

def get_video_length(filename):

    output = subprocess.check_output(("ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", filename)).strip()
    video_length = int(float(output))
    print ("Video length in seconds: "+str(video_length))

    return video_length

def ceildiv(a, b):
    return int(math.ceil(a / float(b)))

def split_by_seconds(filename, split_length, vcodec="copy", acodec="copy",
                     extra="", video_length=None, **kwargs):
    if split_length and split_length <= 0:
        print("Split length can't be 0")
        raise SystemExit

    if not video_length:
        video_length = get_video_length(filename)
    split_count = ceildiv(video_length, split_length)
    if(split_count == 1):
        print("Video length is less then the target split length.")
        raise SystemExit

#     split_cmd = ["ffmpeg", "-i", filename, "-vcodec", vcodec, "-acodec", acodec] + shlex.split(extra)
    split_cmd = ["ffmpeg"] + shlex.split(extra)
    try:
        filebase = ".".join(filename.split(".")[:-1])
        shortname = filebase.split('/')[-1]
        fileext = filename.split(".")[-1]
    except IndexError as e:
        raise IndexError("No . in filename. Error: " + str(e))
    
    path = "vid_short/" + shortname
    cleardir(path)
        
    for n in range(0, split_count):
        split_args = []
        if n == 0:
            split_start = 0
        else:
            split_start = split_length * n

#         split_args += ["-ss", str(split_start), "-t", str(split_length),
#                        filebase + "-" + str(n+1) + "-of-" + \
#                         str(split_count) + "." + fileext]
        split_args += ["-ss", str(split_start), "-t", str(split_length), "-i", filename, 
                       "-vcodec", vcodec, "-acodec", acodec,
                       "vid_short/" + shortname + "/" + shortname + str(n+1) + "." + fileext]
        print ("About to run: "+" ".join(split_cmd+split_args))
        subprocess.check_output(split_cmd+split_args)

In [33]:
# https://www.geeksforgeeks.org/python-program-extract-frames-using-opencv/
# Program To Read video 
# and Extract Frames 
  
# Function to extract frames 
def FrameCapture(inpath, outfolder, div = 24): 
      
    cleardir(outfolder)
    
    # Path to video file 
    vidObj = cv2.VideoCapture(inpath) 
  
    # Used as counter variable 
    count = 0
  
    # checks whether frames were extracted 
    success = 1
  
    while success: 
        vidObj.set(cv2.CAP_PROP_POS_MSEC,(count*1000/div))
        # vidObj object calls read 
        # function extract frames 
        success, image = vidObj.read() 
        if not success:
            break
        # Saves the frames with frame-count 
        cv2.imwrite((outfolder + "/" + "frame%d.jpg") % count, image) 
  
        count += 1
  


In [45]:
def rejoin(folder, pattern, outfile, fps):
    cmd = "ffmpeg -r " + str(fps) + " -i " + folder + '/' + pattern + " -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p " + outfile
    
    print(cmd)
    os.system(cmd)
#     os.system("ffmpeg -r 1 -i frame%01d.jpg -vcodec mpeg4 -y movie.mp4")

In [28]:
def edge_promoting(root, save):
    file_list = os.listdir(root)
    if not os.path.isdir(save):
        os.makedirs(save)
    kernel_size = 5
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    gauss = cv2.getGaussianKernel(kernel_size, 0)
    gauss = gauss * gauss.transpose(1, 0)
    n = 1
    for f in tqdm(file_list):
        rgb_img = cv2.imread(os.path.join(root, f))
        gray_img = cv2.imread(os.path.join(root, f), 0)
        rgb_img = cv2.resize(rgb_img, (256, 256))
        pad_img = np.pad(rgb_img, ((2,2), (2,2), (0,0)), mode='reflect')
        gray_img = cv2.resize(gray_img, (256, 256))
        edges = cv2.Canny(gray_img, 100, 200)
        dilation = cv2.dilate(edges, kernel)

        gauss_img = np.copy(rgb_img)
        idx = np.where(dilation != 0)
        for i in range(np.sum(dilation != 0)):
            gauss_img[idx[0][i], idx[1][i], 0] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 0], gauss))
            gauss_img[idx[0][i], idx[1][i], 1] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 1], gauss))
            gauss_img[idx[0][i], idx[1][i], 2] = np.sum(np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 2], gauss))

        result = np.concatenate((rgb_img, gauss_img), 1)

        cv2.imwrite(os.path.join(save, str(n) + '.png'), result)
        n += 1

In [38]:
split_start = 0
split_length = 10
outfile = "JaredShort"
filename = "AnimeGAN/vid_data/full/Jared.mp4"
shortoutfolder = "baseline/short"
frameoutfolder = "baseline/frames_in" + "/" + outfile
frameoutfolder2 = "baseline/frames_out" + "/" + outfile
videooutfolder = "baseline/merge/" + outfile



# split_args = ["ffmpeg", "-y", "-ss", str(split_start), "-t", str(split_length), "-i", filename, 
#                        "-vcodec", "copy", "-acodec", "copy",
#                        shortoutfolder + "/" + outfile + ".mp4"]
# print ("About to run: "+" ".join(split_args))
# subprocess.check_output(split_args)

# FrameCapture(shortoutfolder + "/" + outfile + ".mp4", frameoutfolder)

split_args = ["python", "CartoonGAN-Pytorch/test.py", "--input_dir", frameoutfolder, "--output_dir",
             frameoutfolder2, "--gpu", "0", "--model_path", "CartoonGAN-Pytorch/pretrained_model"]
cleardir(frameoutfolder2)
print ("About to run: "+" ".join(split_args))
subprocess.check_output(split_args)

Successfully created the directory baseline/frames_out/JaredShort 
About to run: python CartoonGAN-Pytorch/test.py --input_dir baseline/frames_in/JaredShort --output_dir baseline/frames_out/JaredShort --gpu 0 --model_path CartoonGAN-Pytorch/pretrained_model


b'GPU mode\nFinished!\n'

In [47]:
videooutfolder = "baseline/merge/" + outfile
cleardir(videooutfolder)
rejoin(frameoutfolder2, "frame%01d_Hayao.jpg", videooutfolder + "/" + "Hayao.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Hosoda.jpg", videooutfolder + "/" + "Hosoda.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Shinkai.jpg", videooutfolder + "/" + "Shinkai.mp4" ,24)
rejoin(frameoutfolder2, "frame%01d_Paprika.jpg", videooutfolder + "/" + "Paprika.mp4" ,24)

Successfully created the directory baseline/merge/JaredShort 
ffmpeg -r 24 -i baseline/frames_out/JaredShort/frame%01d_Hayao.jpg -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p baseline/merge/JaredShort/Hayao.mp4
ffmpeg -r 24 -i baseline/frames_out/JaredShort/frame%01d_Hosoda.jpg -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p baseline/merge/JaredShort/Hosoda.mp4
ffmpeg -r 24 -i baseline/frames_out/JaredShort/frame%01d_Shinkai.jpg -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p baseline/merge/JaredShort/Shinkai.mp4
ffmpeg -r 24 -i baseline/frames_out/JaredShort/frame%01d_Paprika.jpg -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p baseline/merge/JaredShort/Paprika.mp4


In [48]:
rejoin(frameoutfolder2, "frame%01d_Shinkai.jpg", videooutfolder + "/" + "ShinkaiSlow.mp4" ,2)

ffmpeg -r 2 -i baseline/frames_out/JaredShort/frame%01d_Shinkai.jpg -vcodec mpeg4 -y -c:v libx264 -pix_fmt yuv420p baseline/merge/JaredShort/ShinkaiSlow.mp4
